In [1]:
from osgeo import gdal,ogr,osr
import os
import numpy as np
import matplotlib.pyplot as plt
#import spectral
#import rasterio
#import spectral.io.envi as envi
#import json
import math
from math import sqrt
import pandas as pd
import geopandas as gpd
import affine
from affine import Affine
import helper_functions

In [2]:
# Path to the external SSD
# external_ssd_path = '/Volumes/Desk SSD'
external_ssd_path = '/Volumes/T9'
source_folder_name = "sensor_validation/"
source_folder_path = os.path.join(external_ssd_path,source_folder_name)
subzone = "subzone_c"
source_folder_subzones = ["subzone_c","subzone_d","subzone_e"]

# Check if the external SSD is mounted
if os.path.exists(external_ssd_path):
    print(f"External SSD mounted at: {external_ssd_path}")

External SSD mounted at: /Volumes/T9


In [3]:
flightstrips, hsi_img_object, plot_site_shapefile = helper_functions.get_subzone_objects(source_folder_path, subzone)

HSI Source folder path: /Volumes/T9/sensor_validation/subzone_c/hsi
Plots Source folder path: /Volumes/T9/sensor_validation/subzone_c/plots


In [4]:
hsi_img_object

{'ang20170709t224839rfl': <osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x103d09e00> >,
 'ang20170714t235124rfl': <osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x103d08510> >,
 'ang20190706t234547rfl': <osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x103d09f50> >,
 'ang20190706t235120rfl': <osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x103d09fe0> >,
 'ang20190712t212208rfl': <osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x103d0a5e0> >,
 'ang20190712t212749rfl': <osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x103d08690> >}

In [5]:
for flightstrip in flightstrips:
    # read a few key properties of the image
    nbands = hsi_img_object[flightstrip].RasterCount
    ncols = hsi_img_object[flightstrip].RasterXSize
    nrows = hsi_img_object[flightstrip].RasterYSize

    print("\n".join(["Flightstrip:\t"+str(flightstrip),"Bands:\t"+str(nbands),"Cols (x-axis):\t"+str(ncols),"Rows (y-axis):\t"+str(nrows)]))
    print("-------------------------------------")

Flightstrip:	ang20170709t224839rfl
Bands:	425
Cols (x-axis):	655
Rows (y-axis):	3883
-------------------------------------
Flightstrip:	ang20170714t235124rfl
Bands:	425
Cols (x-axis):	652
Rows (y-axis):	3614
-------------------------------------
Flightstrip:	ang20190706t234547rfl
Bands:	425
Cols (x-axis):	1041
Rows (y-axis):	3912
-------------------------------------
Flightstrip:	ang20190706t235120rfl
Bands:	425
Cols (x-axis):	647
Rows (y-axis):	3798
-------------------------------------
Flightstrip:	ang20190712t212208rfl
Bands:	425
Cols (x-axis):	957
Rows (y-axis):	3837
-------------------------------------
Flightstrip:	ang20190712t212749rfl
Bands:	425
Cols (x-axis):	662
Rows (y-axis):	3751
-------------------------------------


In [7]:
# Get the layer from the shapefile
layer = plot_site_shapefile.GetLayer()

# Create an empty list to store feature attributes
attribute_list = []

# Get field names (attribute names)
layer_definition = layer.GetLayerDefn()
field_names = [layer_definition.GetFieldDefn(i).GetName() for i in range(layer_definition.GetFieldCount())]

# Iterate over the features in the shapefile
for feature in layer:
    # Extract the attribute values for the feature
    attributes = [feature.GetField(field) for field in field_names]
    attribute_list.append(attributes)

# Create a pandas DataFrame with the extracted attributes
df = pd.DataFrame(attribute_list, columns=field_names)

# Show the DataFrame (tabular format of the shapefile attributes)
print(df)

# Optionally, display the first few rows
#df.head()

  C2_plot_jp Field_rele Modified_r              Date__year  \
0       None       1304       1304  1972, 1999, 2008, 2024   
1       None       1303       1303  1972, 1999, 2008, 2023   
2       None        208        208  1972, 1999, 2008, 2011   
3       None        206        206  1972, 1999, 2008, 2010   

               Year_plot_                                           Dataset  \
0  1972, 1999, 2008, 2024  Barrow IBP (Webber 1978, Villarreal et al. 2012)   
1  1972, 1999, 2008, 2023  Barrow IBP (Webber 1978, Villarreal et al. 2012)   
2  1972, 1999, 2008, 2011  Barrow IBP (Webber 1978, Villarreal et al. 2012)   
3  1972, 1999, 2008, 2010  Barrow IBP (Webber 1978, Villarreal et al. 2012)   

                                          Surveyed_b       lat        lon  \
0  Patrick Webber, Sandra Villarreal & Craig Tweedie  71.29462  -156.6882   
1  Patrick Webber, Sandra Villarreal & Craig Tweedie  71.29456  -156.6891   
2  Patrick Webber, Sandra Villarreal & Craig Tweedie  71.29430

In [8]:
plot_strips = False

if plot_strips:
    # Number of images
    num_images = len(hsi_img_object)

    # Define the number of rows and columns for the subplot grid (e.g., 2 rows, 3 columns for 6 images)
    n_cols = 3  # Adjust this based on how you want to display them
    n_rows = (num_images + n_cols - 1) // n_cols  # Calculate required rows

    # Create subplots
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 10))  # Adjust figsize to fit all images

    # Flatten the axes array if there's more than 1 row
    axes = axes.flatten()

    # Iterate over folder names and GDAL objects in the dictionary
    for idx, (folder_name, gdal_object) in enumerate(hsi_img_object.items()):
        if gdal_object:
            # Read the 57th band (red band)
            img_red = gdal_object.GetRasterBand(57).ReadAsArray()

            # Plot the image in the corresponding subplot
            ax = axes[idx]
            ax.imshow(img_red, vmin=0, vmax=0.1)
            ax.set_title(f"{folder_name}", fontsize=8)
            ax.set_xticks([])
            ax.set_yticks([])

            # Add a colorbar to each subplot (optional)
            fig.colorbar(ax.imshow(img_red, vmin=0, vmax=0.1), ax=ax)
        else:
            ax.set_visible(False)  # Hide empty subplots if there are fewer images than subplots

    # Adjust layout to prevent overlapping titles and colorbars
    plt.tight_layout()

    # Show the plot with all images
    plt.show()


In [9]:
flightstrip_transformation = {}

for flightstrip in flightstrips:
    # Print the EPSG code
    hs_crs = helper_functions.return_hsi_projection(hsi_img_object[flightstrip])
    #print(f"EPSG Code Hyperspectral Image {flightstrip}: {hs_crs}")
    # shapefiles have a nested structure: layer(s) -> feature(s) -> geometry
    
    lyr = plot_site_shapefile.GetLayer() # get the only layer in the shapefile
    feat = lyr.GetFeature(1)          # get the first feature in the layer (1 feature per site)
    geom = feat.GetGeometryRef()      # get the feature's geometry

    # get transform for decimal degrees
    from_srs = lyr.GetSpatialRef()                                         # get shapefile srs def
    to_srs = osr.SpatialReference()                                        # init ogr srs object
    to_srs.ImportFromEPSG(int(hs_crs))                                     # import wgs84 srs def
    xytransform = osr.CoordinateTransformation(from_srs,to_srs)            # get transform object

    
    # get UTM and lat/long coordinates for each of the sites
    utm_coordinate_pairs = {}
    ll_coordinate_pairs = {}

    for feature in lyr:
        geom = feature.GetGeometryRef()                                    # get site geometry
        utm_coordinate_pairs[feature['Modified_r']] = (geom.GetX(), geom.GetY()) # get x,y utm coordinates 
        geom.Transform(xytransform)                                        # to wgs84
        ll_coordinate_pairs[feature['Modified_r']] = (geom.GetX(), geom.GetY())  # get lon, lat

    affine_transform = affine.Affine.from_gdal(*hsi_img_object[flightstrip].GetGeoTransform())     # affine forward transform
    inverse_transform = ~affine_transform

    plot_site_pixel_coordinates = pd.DataFrame(columns = ["Subzone", "Flightstrip", "Modified_r", "lon", "lat", "pixel_x", "pixel_y"])

    for site, coords in ll_coordinate_pairs.items():
        lon, lat = coords
        px, py = inverse_transform * (lon, lat)                                    # apply to x,y coordinates
        px, py = round(px), round(py)                                  # get new x,y as integers
        plot_site_pixel_coordinates = plot_site_pixel_coordinates._append({"Subzone" : subzone, "Flightstrip" : flightstrip, "Modified_r" : site, "lon" : lon, "lat" : lat, "pixel_x" : px, "pixel_y" : py},
            ignore_index = True)

    flightstrip_transformation[flightstrip] = {
        "utm_coordinate_pairs": utm_coordinate_pairs,
        "ll_coordinate_pairs": ll_coordinate_pairs,
        "plot_site_pixel_coordinates": plot_site_pixel_coordinates
    }

/var/folders/3m/0hcxmtys0jj01y_svh59k5fm0000gn/T/ipykernel_6431/90617595.py:39: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  plot_site_pixel_coordinates = plot_site_pixel_coordinates._append({"Subzone" : subzone, "Flightstrip" : flightstrip, "Modified_r" : site, "lon" : lon, "lat" : lat, "pixel_x" : px, "pixel_y" : py},
/var/folders/3m/0hcxmtys0jj01y_svh59k5fm0000gn/T/ipykernel_6431/90617595.py:39: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  plot_site_pixel_coordinates = plot_site_pixel_coord

In [10]:
flightstrip_transformation["ang20170709t224839rfl"]["plot_site_pixel_coordinates"]

,Subzone,Flightstrip,Modified_r,lon,lat,pixel_x,pixel_y
0,subzone_c,ang20170709t224839rfl,1304,582729.601538,7.911838e+06,186,1107
1,subzone_c,ang20170709t224839rfl,1303,582697.923456,7.911830e+06,181,1104
2,subzone_c,ang20170709t224839rfl,208,582792.965712,7.911804e+06,191,1121
3,subzone_c,ang20170709t224839rfl,206,582783.561674,7.911846e+06,195,1113


In [ ]:
for flightstrip in flightstrips:
    print(flightstrip_transformation[flightstrip]["plot_site_pixel_coordinates"])
    print("--------------------------------------------")

In [ ]:
for flightstrip in flightstrips:
    # Extracting metadata keys and values
    metadata_keys = hsi_img_object[flightstrip].GetMetadata().keys()
    metadata_values = hsi_img_object[flightstrip].GetMetadata().values()


    # Lists of band numbers and band centers
    band_numbers = [
        int(key.split("_")[1])
        for key in metadata_keys
        if key != "wavelength_units"
    ]

    band_centers = [
        float(value.split(" ")[0])
        for value in metadata_values
        if value != "Nanometers"
    ]

    # Classify each band center
    em_regions = [helper_functions.classify_band(b) for b in band_centers]


    # get the reflectance values of the site (careful: these values are already sorted --> add to df after sort)
    #testsite1_reflectance = img_open.ReadAsArray(px1,py1,1,1).flatten().tolist()
    #testsite2_reflectance = img_open.ReadAsArray(px2,py2,1,1).flatten().tolist()


    # Function to get reflectance for each site
    def get_site_reflectance(px, py):
        reflectance = hsi_img_object[flightstrip].ReadAsArray(px,py,1,1).flatten()
        return reflectance

    # Apply the function to each row and add the result as a new column
    flightstrip_transformation[flightstrip]["plot_site_pixel_coordinates"]["reflectance"] = flightstrip_transformation[flightstrip]["plot_site_pixel_coordinates"].apply(lambda row: get_site_reflectance(row["pixel_x"], row["pixel_y"]), axis=1)


In [ ]:
for flightstrip in flightstrips:
    print(flightstrip_transformation[flightstrip]["plot_site_pixel_coordinates"])

print(flightstrip_transformation["ang20190712t212749rfl"]["plot_site_pixel_coordinates"]["reflectance"])
print(flightstrip_transformation["ang20190712t212749rfl"]["plot_site_pixel_coordinates"])
print(flightstrip_transformation["ang20190712t212749rfl"])

In [ ]:
# Initialize an empty list to store the DataFrames
combined_dataframes = []

# Loop through the list and accumulate the DataFrames
for flightstrip in flightstrips:
    combined_dataframes.append(flightstrip_transformation[flightstrip]["plot_site_pixel_coordinates"])

# Concatenate all DataFrames at once
combined_df = pd.concat(combined_dataframes, ignore_index=True)

combined_df = combined_df.sort_values(by=['Modified_r'])

combined_df.head(25)

In [ ]:
unique_plotsite_identifier_values = combined_df['Modified_r'].unique()

In [ ]:
dataframe_array = {}

# DataFrame describing bands
bands = pd.DataFrame({
    "Band number": band_numbers,
    "Band center (nm)": band_centers,
    "EM region": em_regions
}, index=band_numbers).sort_index()

for plotsite in unique_plotsite_identifier_values:
    df_location = bands
    # Iterate over rows
    for row in combined_df[combined_df['Modified_r'] == plotsite].itertuples():
        df_location[f"{row.Flightstrip}_Reflectance"] = row.reflectance

    dataframe_array[f'df_{plotsite}'] = df_location

### Create a reflectance plot for the 2 test sites

It can be seen that in the region of arround 1300 nanometer and towards the end we have a lot of noise. In the next step this will be removed manualy.

### Create a plot with noise bands removed

In [ ]:
bblist = np.ones((425,))
bblist[0:14] = 0
bblist[189:225] = 0
bblist[281:336] = 0
bblist[405:] = 0

titlefont = {'fontsize':16,'fontweight':2,
             'verticalalignment':'baseline','horizontalalignment':'center'}
plt.rcParams['figure.figsize'] = [16, 8]

for plotsite in dataframe_array:
    for series_name, series in dataframe_array[plotsite].items():
        if(series_name == "Band center (nm)"):
            wavelength_flightstrip = series
            #print(series)
        elif("_Reflectance" in series_name):
            pixel_flightstrip = dataframe_array[plotsite][series_name]
            pixel_flightstrip = np.reshape(pixel_flightstrip,(425))
            pixel_flightstrip[bblist == 0] = np.nan
            pixel_flightstrip[pixel_flightstrip==-9999] = np.nan
            plt.plot(wavelength_flightstrip, pixel_flightstrip,  label = series_name)
            #print(f"Plotsite {plotsite} flightstrip {series_name}")
        
        plt.xlabel ( 'Wavelength [nm]', fontsize=20)
        #plt.ylim(y_min, y_max)
        plt.ylabel('Reflectance', fontsize=20)
        plt.title(plotsite, fontdict = titlefont, pad = 10)
        plt.grid()
        plt.legend(loc='upper left', bbox_to_anchor=(1, 1), title='Sites', title_fontsize='13', fontsize='11', borderaxespad=0.)

    plt.show()




In [ ]:
dataframe_array

In [ ]:
# remove a few band to reduce atmospheric noise 
wavelengths = bands["Band center (nm)"]
bblist = np.ones((425,))
bblist[0:14] = 0
bblist[189:225] = 0
bblist[281:336] = 0
bblist[405:] = 0


titlefont = {'fontsize':16,'fontweight':2,
             'verticalalignment':'baseline','horizontalalignment':'center'}
plt.rcParams['figure.figsize'] = [16, 8]



In [ ]:
pixel = dataframe_array["df_1304"]["ang20170709t224839rfl_Reflectance"]
pixel = np.reshape(pixel,(425))
pixel[bblist == 0] = np.nan
plt.plot(wavelengths, pixel,  label = "df_1304")

In [ ]:
pixel

In [ ]:
# Iterate over rows
for row in plot_site_pixel_coordinates.itertuples():
    pixel = img_open.ReadAsArray(row.pixel_x, row.pixel_y, 1, 1)
    pixel = np.reshape(pixel,(425))
    pixel[bblist == 0] = np.nan
    plt.plot(wavelengths, pixel,  label = row.Modified_r)

plt.xlabel ( 'Wavelength [nm]', fontsize=20)
#plt.ylim(y_min, y_max)
plt.ylabel('Reflectance', fontsize=20)
plt.title("Test sites reflectance", fontdict = titlefont, pad = 10)
plt.grid()
plt.legend(loc='upper left', bbox_to_anchor=(1, 1), title='Sites', title_fontsize='13', fontsize='11', borderaxespad=0.)


plt.show()

### Create the same plot with wavelength indicators

In [ ]:
%matplotlib inline
titlefont = {'fontsize':16,'fontweight':2,
             'verticalalignment':'baseline','horizontalalignment':'center'}
plt.rcParams['figure.figsize'] = [16, 8]

# Iterate over rows
for row in plot_site_pixel_coordinates.itertuples():
    pixel = img_open.ReadAsArray(row.pixel_x, row.pixel_y, 1, 1)
    pixel = np.reshape(pixel,(425))
    pixel[bblist == 0] = np.nan
    plt.plot(wavelengths, pixel,  label = row.Modified_r)


# add shaders for band regions
for region,limits in band_dictionary.items():
    plt.axvspan(limits['lower'], limits['upper'], alpha=0.7, color=limits['color'], label=region)
        
# add water vapor region shaders
plt.axvspan(1320, 1500, alpha=0.2, color='blue', label='water vapor regions')
plt.axvspan(1780, 2055, alpha=0.2, color='blue')
    
# some plotting config
#plt.ylim(y_min,y_max)
#plt.xlim(min(wavelengths),max(wavelengths))
plt.ylabel("reflectance", fontsize=16)
plt.xlabel("wavelength [nm]", fontsize=16)
plt.tick_params(axis='both', which='major', labelsize=14)
plt.grid('on', alpha=0.4)
plt.title("Test sites reflectance", fontdict = titlefont, pad = 10)

# now add custom legend
plt.legend(loc='upper left', bbox_to_anchor=(1, 1), title='Sites', title_fontsize='13', fontsize='11', borderaxespad=0.)